In [1]:
!nvidia-smi

Wed Jul 14 12:01:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Faster RCNN + InceptionResNet **V2**

## **Imports**

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import tempfile
import time
import pathlib

from six.moves.urllib.request import urlopen
from six import BytesIO
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

print(tf.__version__)

2.5.0


## Helper Functions

In [3]:
def load_imgs(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img


def run_detector(detector, img):

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  result = detector(converted_img)

  result = {key:value.numpy() for key,value in result.items()}

  return result["detection_boxes"],result["detection_class_entities"],result["detection_scores"]


def prepare_RCNN_model(path):
    return hub.load(path).signatures['default']


# Deep MAC

## Install requerments

In [4]:
!pip install -U --pre tensorflow=="2.2.0"
!pip install tensorflow-object-detection-api

  Using cached https://files.pythonhosted.org/packages/4c/1a/0d79814736cfecc825ab8094b39648cc9c46af7af1bae839928acb73b4dd/tensorflow-2.2.0-cp37-cp37m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d6/84/759f5dd23fec8ba71952d97bcc7e2c9d7d63bdc582421f3cd4be845f0c98/gast-0.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a4/f5/926ae53d6a226ec0fda5208e0e581cffed895ccc89e36ba76a8e60895b78/tensorflow_estimator-2.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3f/c0/abde58b837e066bca19a3f7332d9d0493521d7dd6b48248451a9e3fe2214/h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/1d/74/0a6fcb206dcc72a6da9a62dd81784bfdbff5fedb099982861dc2219014fb/tensorboard-2.2.2-py3-none-any.whl
ERROR: tf-models-official 2.5.0 has requirement tensorflow>=2.5.0, but you'll have tensorflow 2.2.0 which is incompatible.
  Found existing installation: gast 0.4.0
    Uninstal

## Clone the tensorflow models repository

In [5]:
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Install the Object Detection API

In [6]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Using cached https://files.pythonhosted.org/packages/aa/fd/993aa1333eb54d9f000863fe8ec61e41d12eb833dea51484c76c038718b5/tensorflow-2.5.0-cp37-cp37m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/9d/74/9eae2bedd8201ab464308f42c601a12d79727a1c87f0c867fdefb212c6cf/h5py-3.1.0-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ec/78/b27f73e923becc6e79e18fe112cf75e3200d1ee35b0dba8fa46181bce56c/tensorflow_estimator-2.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b6/48/583c032b79ae5b3daa02225a675aeb673e58d2cb698e78510feceb11958c/gast-0.4.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/44/f5/7feea02a3fb54d5db827ac4b822a7ba8933826b36de21880518250b8733a/tensorboard-2.5.0-py3-none-any.whl
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1658497 sha256=16550b802ea79a42ca94cbd6f3ae91f15e33e053

## Imports

In [7]:
import glob
import io
import logging
import os
import random
import warnings

import gc
from matplotlib.backends.backend_agg import FigureCanvas
from matplotlib.figure import Figure

import imageio
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage
import matplotlib
from matplotlib import patches
import matplotlib.pyplot as plt
import numpy as np
from object_detection.utils import colab_utils
from object_detection.utils import ops
from object_detection.utils import visualization_utils as viz_utils
from PIL import Image, ImageDraw, ImageFont
import scipy.misc
from six import BytesIO
from skimage import color
from skimage import transform
from skimage import util
from skimage.color import rgb_colors
import tensorflow as tf

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

In [8]:
%matplotlib inline

COLORS = ([rgb_colors.cyan, rgb_colors.orange, rgb_colors.pink,
           rgb_colors.purple, rgb_colors.limegreen , rgb_colors.crimson] +
          [(color) for (name, color) in color.color_dict.items()])
random.shuffle(COLORS)

logging.disable(logging.WARNING)

## Helper Functions

In [9]:
def resize_for_display(image, max_height=600):
  height, width, _ = image.shape
  width = int(width * max_height / height)
  with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    return util.img_as_ubyte(transform.resize(image, (height, width)))


def get_mask_prediction_function(model):
  """Get single image mask prediction function using a model."""

  @tf.function
  def predict_masks(image, boxes):
    height, width, _ = image.shape.as_list()
    batch = image[tf.newaxis]
    boxes = boxes[tf.newaxis]

    detections = model(batch, boxes)
    masks = detections['detection_masks']

    return ops.reframe_box_masks_to_image_masks(masks[0], boxes[0],
                                                height, width)
    
  return predict_masks



def plot_image_annotations(image, boxes, masks, darken_image=0.5):
    fig = Figure()
    canvas = FigureCanvas(fig)
    ax = fig.subplots()
    ax.set_axis_off()

    image = image.numpy()

    image = (image * darken_image).astype(np.uint8)
    ax.imshow(image)

    height, width, _ = image.shape

    num_colors = len(COLORS)
    color_index = 0

    for box, mask in zip(boxes, masks):
        ymin, xmin, ymax, xmax = box
        ymin *= height
        ymax *= height
        xmin *= width
        xmax *= width

        color = COLORS[color_index]
        color = np.array(color)
        rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                    linewidth=2.5, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
        mask = (mask > 0.5).numpy().astype(np.uint8)
        color_image = np.ones_like(image) * color[np.newaxis, np.newaxis, :]
        color_and_mask = np.concatenate(
            [color_image, mask[:, :, np.newaxis]], axis=2)

        ax.imshow(color_and_mask, alpha=0.5)

        color_index = (color_index + 1) % num_colors

    canvas.draw()
    ax = np.array(canvas.renderer.buffer_rgba())

    return ax.astype(np.uint8)


def prepare_model(model_path):

    print('Downloading and untarring model')
    !wget http://download.tensorflow.org/models/object_detection/tf2/20210329/deepmac_1024x1024_coco17.tar.gz
    !cp deepmac_1024x1024_coco17.tar.gz models/research/object_detection/test_data/
    !tar -xzf models/research/object_detection/test_data/deepmac_1024x1024_coco17.tar.gz
    !mv deepmac_1024x1024_coco17 models/research/object_detection/test_data/

    print('Loading SavedModel')
    model = tf.keras.models.load_model(model_path)
    prediction_function = get_mask_prediction_function(model)

    return prediction_function


def best_boxes(boxes,scores,max_boxes=10, min_score=0.1):
    boxes = boxes[0:len(scores[scores > min_score])]
    num_boxes = min(boxes.shape[0], max_boxes)
    return boxes[0:num_boxes]


def run_app(image,detector,prediction_function,max_boxes=10,min_score=0.1,darken_image=0.5):

    image = tf.convert_to_tensor(np.array(image), dtype='uint8')

    boxes ,detection_class_entities ,detection_scores = run_detector(detector, image)
    boxes = best_boxes(boxes,detection_scores,max_boxes,min_score)


    masks = prediction_function(tf.convert_to_tensor(image),
                                tf.convert_to_tensor(boxes, dtype=tf.float32))
    new_frame = plot_image_annotations(image, boxes, masks, darken_image=darken_image)

    return new_frame


In [ ]:
path = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
detector = prepare_RCNN_model(path)

path = 'models/research/object_detection/test_data/deepmac_1024x1024_coco17/saved_model'
prediction_function = prepare_model(path)


video_reader = imageio.get_reader('/content/pexels-kelly-lacy-5473765.mp4')
video_writer = imageio.get_writer('/content/mp4_new.mp4')

start = time.time()
for image in video_reader:

    new_frame = run_app(image,detector,prediction_function,max_boxes=10,min_score=0.3,darken_image=0.5)

    del image
    gc.collect()

    video_writer.append_data(new_frame)
end = time.time()
print(f"Runtime of the program is {end - start}")

video_writer.close()

--2021-07-14 12:07:28--  http://download.tensorflow.org/models/object_detection/tf2/20210329/deepmac_1024x1024_coco17.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.195.128, 2607:f8b0:400e:c02::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.195.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1792904062 (1.7G) [application/x-tar]
Saving to: ‘deepmac_1024x1024_coco17.tar.gz.1’

deepmac_1024x1024_c 100%[===================>]   1.67G  39.3MB/s    in 12s     

2021-07-14 12:07:41 (141 MB/s) - ‘deepmac_1024x1024_coco17.tar.gz.1’ saved [1792904062/1792904062]

mv: cannot move 'deepmac_1024x1024_coco17' to 'models/research/object_detection/test_data/deepmac_1024x1024_coco17': Directory not empty
Loading SavedModel
